In [1]:
## generate mesh

import svgwrite
import random

def generate_polygon_mesh_svg(file_path, width=1800, height=1200, cols=30, rows=20):
    """Generate a polygon mesh SVG with randomized 3-4 sided polygons."""
    dwg = svgwrite.Drawing(file_path, size=(width, height))
    
    base_w = width // cols
    base_h = height // rows
    
    # Store generated points
    points = [[(i * base_w + random.uniform(-base_w * 0.25, base_w * 0.25), 
                j * base_h + random.uniform(-base_h * 0.25, base_h * 0.25)) for j in range(rows+1)] for i in range(cols+1)]
    
    # Generate polygonal connections using line elements
    for i in range(cols):
        for j in range(rows):
            p1 = points[i][j]
            p2 = points[i+1][j]
            p3 = points[i+1][j+1]
            p4 = points[i][j+1]

            if random.random() > 0.5:
                # Draw quadrilateral
                dwg.add(dwg.line(p1, p2, stroke="black", stroke_width=1))
                dwg.add(dwg.line(p2, p3, stroke="black", stroke_width=1))
                dwg.add(dwg.line(p3, p4, stroke="black", stroke_width=1))
                dwg.add(dwg.line(p4, p1, stroke="black", stroke_width=1))
            else:
                # Draw two triangles instead of a quadrilateral
                dwg.add(dwg.line(p1, p2, stroke="black", stroke_width=1))
                dwg.add(dwg.line(p2, p3, stroke="black", stroke_width=1))
                dwg.add(dwg.line(p3, p1, stroke="black", stroke_width=1))

                dwg.add(dwg.line(p1, p3, stroke="black", stroke_width=1))
                dwg.add(dwg.line(p3, p4, stroke="black", stroke_width=1))
                dwg.add(dwg.line(p4, p1, stroke="black", stroke_width=1))
    
    dwg.save()

# Example usage
output_file = "polygon_mesh.svg"
generate_polygon_mesh_svg(output_file)

In [ ]:
# match mesh points to shape

import svgwrite
import numpy as np
from lxml import etree as ET
from scipy.spatial import KDTree

def extract_svg_endpoints(svg_path):
    """Extracts all unique endpoints from line elements in an SVG."""
    tree = ET.parse(svg_path)
    root = tree.getroot()
    ns = {'svg': 'http://www.w3.org/2000/svg'}
    
    endpoints = set()
    
    for line in root.findall('.//svg:line', ns):
        x1, y1 = float(line.get('x1', 0)), float(line.get('y1', 0))
        x2, y2 = float(line.get('x2', 0)), float(line.get('y2', 0))
        endpoints.add((x1, y1))
        endpoints.add((x2, y2))
    
    return list(endpoints)

def adjust_mesh_to_shape(mesh_svg, shape_svg, output_svg, threshold=10):
    """Moves mesh nodes to the closest matching nodes from the shape SVG."""
    mesh_tree = ET.parse(mesh_svg)
    mesh_root = mesh_tree.getroot()
    ns = {'svg': 'http://www.w3.org/2000/svg'}
    
    mesh_points = extract_svg_endpoints(mesh_svg)
    shape_points = extract_svg_endpoints(shape_svg)
    
    if not shape_points:
        print("No valid shape points found. Exiting.")
        return
    
    shape_tree = KDTree(shape_points)
    updated_positions = {}
    
    for pt in mesh_points:
        dist, index = shape_tree.query(pt)
        if dist < threshold:
            updated_positions[pt] = shape_points[index]
    
    for line in mesh_root.findall('.//svg:line', ns):
        x1, y1 = float(line.get('x1', 0)), float(line.get('y1', 0))
        x2, y2 = float(line.get('x2', 0)), float(line.get('y2', 0))
        
        if (x1, y1) in updated_positions:
            x1, y1 = updated_positions[(x1, y1)]
        if (x2, y2) in updated_positions:
            x2, y2 = updated_positions[(x2, y2)]
        
        line.set('x1', str(x1))
        line.set('y1', str(y1))
        line.set('x2', str(x2))
        line.set('y2', str(y2))
    
    mesh_tree.write(output_svg, encoding='utf-8', xml_declaration=True)
    print(f"Output saved to {output_svg}")

# Example usage
mesh_input = "bg_mesh/mesh_2030.svg"
shape_input = "bg_mesh/3.svg"
output_file = "bg_mesh/3_mesh.svg"
adjust_mesh_to_shape(mesh_input, shape_input, output_file, threshold=50)


In [ ]:
import svgwrite
import numpy as np

def generate_sphere_points(subdivisions=3, radius=300):
    """Generates a faceted sphere using icosphere subdivision."""
    phi = (1 + np.sqrt(5)) / 2  # Golden ratio
    vertices = [
        [-1, phi, 0], [1, phi, 0], [-1, -phi, 0], [1, -phi, 0],
        [0, -1, phi], [0, 1, phi], [0, -1, -phi], [0, 1, -phi],
        [phi, 0, -1], [phi, 0, 1], [-phi, 0, -1], [-phi, 0, 1]
    ]
    vertices = np.array(vertices) / np.linalg.norm(vertices, axis=1)[:, np.newaxis]  # Normalize to unit sphere
    vertices = vertices.tolist()  # Convert back to list for modification
    
    faces = [
        (0, 11, 5), (0, 5, 1), (0, 1, 7), (0, 7, 10), (0, 10, 11),
        (1, 5, 9), (5, 11, 4), (11, 10, 2), (10, 7, 6), (7, 1, 8),
        (3, 9, 4), (3, 4, 2), (3, 2, 6), (3, 6, 8), (3, 8, 9),
        (4, 9, 5), (2, 4, 11), (6, 2, 10), (8, 6, 7), (9, 8, 1)
    ]
    
    for _ in range(subdivisions):
        new_faces = []
        midpoint_cache = {}
        
        def get_midpoint(v1, v2):
            """Finds or creates a midpoint between two vertices."""
            key = tuple(sorted((v1, v2)))
            if key in midpoint_cache:
                return midpoint_cache[key]
            midpoint = (np.array(vertices[v1]) + np.array(vertices[v2])) / 2
            midpoint /= np.linalg.norm(midpoint)  # Normalize to unit sphere
            vertices.append(midpoint.tolist())
            midpoint_cache[key] = len(vertices) - 1
            return midpoint_cache[key]
        
        for v1, v2, v3 in faces:
            a = get_midpoint(v1, v2)
            b = get_midpoint(v2, v3)
            c = get_midpoint(v3, v1)
            new_faces.extend([(v1, a, c), (v2, b, a), (v3, c, b), (a, b, c)])
        
        faces = new_faces
    
    vertices = np.array(vertices) * radius
    return vertices, faces

def project_3d_to_2d(points, width=800, height=800, fov=500):
    """Projects 3D points onto a 2D plane using simple perspective projection."""
    projected = []
    for x, y, z in points:
        scale = fov / (fov + z)  # Perspective scaling
        px, py = x * scale + width / 2, -y * scale + height / 2
        projected.append((px, py, z))  # Keep z for back-face culling
    return projected

def should_draw_face(p1, p2, p3):
    """Determines if a face should be drawn using back-face culling."""
    normal = np.cross(np.array(p2) - np.array(p1), np.array(p3) - np.array(p1))
    return normal[2] > 0  # Only draw if normal is facing towards the camera

def generate_faceted_sphere_svg(output_file, subdivisions=3, width=800, height=800):
    """Generates an SVG of a faceted sphere with triangular facets, applying back-face culling."""
    vertices, faces = generate_sphere_points(subdivisions)
    projected_points = project_3d_to_2d(vertices, width, height)
    
    dwg = svgwrite.Drawing(output_file, size=(width, height))
    for v1, v2, v3 in faces:
        p1, p2, p3 = projected_points[v1], projected_points[v2], projected_points[v3]
        if should_draw_face(p1, p2, p3):  # Apply back-face culling
            dwg.add(dwg.line(p1[:2], p2[:2], stroke="black", stroke_width=1))
            dwg.add(dwg.line(p2[:2], p3[:2], stroke="black", stroke_width=1))
            dwg.add(dwg.line(p3[:2], p1[:2], stroke="black", stroke_width=1))
    
    dwg.save()
    print(f"Saved faceted sphere to {output_file}")

# Example usage
output_svg = "faceted_sphere.svg"
generate_faceted_sphere_svg(output_svg, subdivisions=2)


In [11]:
# shapes to lines

from lxml import etree as ET
import re
import numpy as np

def parse_transform(transform):
    """Parses the transform attribute and returns a transformation matrix in correct order."""
    matrix = np.identity(3)
    if not transform:
        return matrix
    
    transform_commands = re.findall(r'(translate|rotate|scale|matrix)\((.*?)\)', transform)
    for command, values in transform_commands:
        values = list(map(float, re.findall(r'-?\d+(?:\.\d+)?', values)))
        
        if command == 'translate' and len(values) >= 1:
            tx, ty = values[0], values[1] if len(values) > 1 else 0
            translate_matrix = np.array([[1, 0, tx], [0, 1, ty], [0, 0, 1]])
            matrix = np.dot(matrix, translate_matrix)  # Correct order
        
        elif command == 'rotate':
            angle = np.radians(values[0])
            cx, cy = (values[1], values[2]) if len(values) == 3 else (0, 0)
            cos_a, sin_a = np.cos(angle), np.sin(angle)
            translate_to_origin = np.array([[1, 0, -cx], [0, 1, -cy], [0, 0, 1]])
            rotate_matrix = np.array([[cos_a, -sin_a, 0], [sin_a, cos_a, 0], [0, 0, 1]])
            translate_back = np.array([[1, 0, cx], [0, 1, cy], [0, 0, 1]])
            matrix = np.dot(matrix, np.dot(translate_to_origin, np.dot(rotate_matrix, translate_back)))
        
        elif command == 'scale' and len(values) >= 1:
            sx, sy = values[0], values[1] if len(values) > 1 else values[0]
            scale_matrix = np.array([[sx, 0, 0], [0, sy, 0], [0, 0, 1]])
            matrix = np.dot(matrix, scale_matrix)
        
        elif command == 'matrix' and len(values) == 6:
            a, b, c, d, e, f = values
            matrix_matrix = np.array([[a, c, e], [b, d, f], [0, 0, 1]])
            matrix = np.dot(matrix, matrix_matrix)
    
    return matrix

def apply_transform(point, matrix):
    """Applies a transformation matrix to a given (x, y) point."""
    x, y = point
    transformed = np.dot(matrix, np.array([x, y, 1]))
    return transformed[0], transformed[1]

def parse_path_d(d, transform_matrix):
    """Extracts M and L commands from a path and applies transformations."""
    commands = re.findall(r'([ML])([^ML]*)', d, re.IGNORECASE)
    points = []
    
    for cmd, values in commands:
        coords = [float(v) for v in re.findall(r'-?\d+(?:\.\d+)?', values)]
        for i in range(0, len(coords), 2):
            points.append(apply_transform((coords[i], coords[i+1]), transform_matrix))
    
    return points

def convert_shapes_to_lines_recursive(element):
    ns = {'svg': 'http://www.w3.org/2000/svg'}
    
    shapes_to_remove = []
    
    for shape in list(element.findall('.//svg:polyline', ns)) + list(element.findall('.//svg:polygon', ns)) + list(element.findall('.//svg:path', ns)):
        shape_type = 'path' if shape.tag.endswith('path') else ('polygon' if shape.tag.endswith('polygon') else 'polyline')
        print(f"Processing {shape_type}: {ET.tostring(shape, pretty_print=True).decode()}")
        
        transform_matrix = parse_transform(shape.attrib.get('transform', ''))
        
        if shape.tag.endswith('path'):
            points_list = parse_path_d(shape.attrib.get('d', ''), transform_matrix)
        else:
            points = shape.attrib.get('points', '').strip()
            if not points:
                print("Skipping empty points attribute")
                continue
            
            flat_points = []
            for p in points.replace(',', ' ').split():
                try:
                    flat_points.append(float(p))
                except ValueError:
                    print(f"Skipping invalid coordinate: {p}")
                    continue
            
            points_list = [apply_transform((flat_points[i], flat_points[i+1]), transform_matrix) for i in range(0, len(flat_points), 2)]
        
        print(f"Parsed points: {points_list}")
        
        if len(points_list) < 2:
            print("Skipping shape with insufficient points")
            continue  # Skip if there's not enough points
        
        parent = shape.getparent()
        if parent is None:
            print("Skipping shape with no parent")
            continue
        
        index = parent.index(shape)
        
        for i in range(len(points_list) - 1):
            x1, y1 = points_list[i]
            x2, y2 = points_list[i + 1]
            line = ET.Element('line', {
                'x1': str(x1), 'y1': str(y1),
                'x2': str(x2), 'y2': str(y2),
                'stroke': shape.attrib.get('stroke', 'black'),
                'stroke-width': shape.attrib.get('stroke-width', '1')
            })
            print(f"Inserting line: {ET.tostring(line, pretty_print=True).decode()}")
            parent.insert(index, line)
            index += 1
        
        shapes_to_remove.append(shape)
    
    for shape in shapes_to_remove:
        print(f"Removing {shape.tag}: {ET.tostring(shape, pretty_print=True).decode()}")
        shape.getparent().remove(shape)
    
    for group in list(element.findall('.//svg:g', ns)):
        convert_shapes_to_lines_recursive(group)

def convert_shapes_to_lines(svg_path, output_path):
    tree = ET.parse(svg_path)
    root = tree.getroot()
    
    convert_shapes_to_lines_recursive(root)
    
    tree.write(output_path, encoding='utf-8', xml_declaration=True)

# Example usage
#convert_shapes_to_lines('input.svg', 'output.svg')

In [ ]:
# Example usage
convert_shapes_to_lines('4_dissolve.svg', '4_dissolve_l.svg')

In [ ]:
# move lines in target (mesh) to positions in source SVG

import xml.etree.ElementTree as ET
import numpy as np
import uuid

def parse_svg_lines(svg_file):
    """Parse SVG file and extract line elements with coordinates."""
    tree = ET.parse(svg_file)
    root = tree.getroot()
    namespace = {'svg': 'http://www.w3.org/2000/svg'}
    
    lines = []
    for line in root.findall(".//svg:line", namespace):
        x1, y1 = float(line.get("x1")), float(line.get("y1"))
        x2, y2 = float(line.get("x2")), float(line.get("y2"))
        lines.append((x1, y1, x2, y2, line))
    
    return tree, root, lines

def add_ids_to_source_lines(tree, lines):
    """Add unique IDs to each line in the source SVG."""
    for line in lines:
        line[-1].set("id", str(uuid.uuid4()))
    tree.write("source_with_ids.svg")

def find_closest_line(target_line, source_lines):
    """Find the closest line in the source SVG based on distance between endpoints."""
    x1_t, y1_t, x2_t, y2_t, _ = target_line
    target_mid = np.array([(x1_t + x2_t) / 2, (y1_t + y2_t) / 2])
    
    min_dist = float("inf")
    closest_source = None
    
    for source_line in source_lines:
        x1_s, y1_s, x2_s, y2_s, _ = source_line
        source_mid = np.array([(x1_s + x2_s) / 2, (y1_s + y2_s) / 2])
        
        dist = np.linalg.norm(target_mid - source_mid)
        
        if dist < min_dist:
            min_dist = dist
            closest_source = source_line
    
    return closest_source

def move_target_lines_to_source(source_svg, target_svg, output_svg):
    """Align target SVG lines to match positions of the closest source SVG lines."""
    source_tree, source_root, source_lines = parse_svg_lines(source_svg)
    target_tree, target_root, target_lines = parse_svg_lines(target_svg)
    
    # Add unique IDs to source lines
    add_ids_to_source_lines(source_tree, source_lines)
    
    for target_line in target_lines:
        closest_source = find_closest_line(target_line, source_lines)
        
        if closest_source:
            x1_s, y1_s, x2_s, y2_s, _ = closest_source
            target_line[-1].set("x1", str(x1_s))
            target_line[-1].set("y1", str(y1_s))
            target_line[-1].set("x2", str(x2_s))
            target_line[-1].set("y2", str(y2_s))
    
    target_tree.write(output_svg)

# Example usage:
# move_target_lines_to_source("source.svg", "target.svg", "aligned_target.svg")

In [ ]:
# create duplicates to random line elements to match a total of num_lines

import xml.etree.ElementTree as ET
import random

def modify_svg(svg_path, num_lines, output_path):
    # Parse the SVG file
    tree = ET.parse(svg_path)
    root = tree.getroot()
    
    # Get the namespace (if any)
    ns = {'svg': 'http://www.w3.org/2000/svg'} if root.tag.startswith('{') else {}
    
    # Find all line elements
    lines = root.findall('.//svg:line', ns) if ns else root.findall('.//line')
    
    if not lines:
        print("No line elements found in the SVG file.")
        return
    
    # Determine how many more lines need to be added
    current_count = len(lines)
    if current_count >= num_lines:
        print("The SVG already contains enough lines.")
        return
    
    additional_lines_needed = num_lines - current_count
    
    for _ in range(additional_lines_needed):
        line_to_duplicate = random.choice(lines)  # Randomly select a line to duplicate
        new_line = ET.Element('line', line_to_duplicate.attrib)  # Copy attributes
        root.append(new_line)
    
    # Save the modified SVG
    tree.write(output_path)
    print(f"SVG modified and saved as {output_path}")

# Example usage
input_svg = "3.svg"
num_lines = 540
output_svg = "3_out.svg"
modify_svg(input_svg, num_lines, output_svg)
